2019 Census Data for Selected Variables - Baltimore City

In [1]:
#pip install us

In [2]:
# pip install censusgeocode
# pip install censusdata

In [3]:
#conda install -c conda-forge cenpy

In [4]:
#conda update -n base -c defaults conda


In [5]:
# From https://cenpy-devs.github.io/cenpy/:
# Cenpy (pronounced sen-pie) is a package that automatically discovers US Census Bureau API endpoints and exposes them to Python in a consistent fashion. 
# It also provides easy-to-use access to certain well-used data products, like the American Community Survey (ACS) and 2010 Decennial Census.
#pip install cenpy

In [6]:
# From https://www.census.gov/programs-surveys/acs/guidance/comparing-acs-data.html:
# "Due to the impact of the COVID-19 pandemic, the Census Bureau changed the 2020 ACS release. 
# Instead of providing the standard 1-year data products, the Census Bureau released experimental estimates from the 1-year data. 
# Data users should not compare 2020 ACS 1-year experimental estimates with any other data.""

In [7]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states
import censusdata 
import censusgeocode as cg
import cenpy
import gmaps
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt

# Census & gmaps API Keys
from config import (api_key, gkey)
c = Census(api_key, year=2019)

# Configure gmaps
#gmaps.configure(api_key=gkey)

C:\Users\Jenkir\anaconda3\envs\PythonData\lib\site-packages\libpysal\weights\util.py:23: UserWarning: geopandas not available. Some functionality will be disabled.
  warn("geopandas not available. Some functionality will be disabled.")


In [8]:
# Set this to false if you're trying to do this without an internet connection
# and data which would have been fetched from an API query will be read from cached files instead
INTERNET_IS_WORKING = True

if INTERNET_IS_WORKING:
    con = cenpy.remote.APIConnection('ACSDT5Y2019')
    variables = con.variables
else:
    variables = pd.read_csv('data/ACSDT5Y2018_variables.csv',index_col='Unnamed: 0')

# No matter which product you use, a cenpy APIConnection can show you the variables 
# which it can retrieve for you. They come back as a pandas DataFrame.
print(f"ACSDT5Y2019 provides {len(variables)} variables.") # how many are there?
variables.head()

ACSDT5Y2019 provides 27080 variables.


,label,concept,predicateType,group,limit,predicateOnly,hasGeoCollectionSupport,attributes,required
for,Census API FIPS 'for' clause,Census API Geography Specification,fips-for,N/A,0,True,NaN,NaN,NaN
in,Census API FIPS 'in' clause,Census API Geography Specification,fips-in,N/A,0,True,NaN,NaN,NaN
ucgid,Uniform Census Geography Identifier clause,Census API Geography Specification,ucgid,N/A,0,True,True,NaN,NaN
B24022_060E,Estimate!!Total:!!Female:!!Service occupations...,SEX BY OCCUPATION AND MEDIAN EARNINGS IN THE P...,int,B24022,0,NaN,NaN,"B24022_060EA,B24022_060M,B24022_060MA",NaN
B19001B_014E,"Estimate!!Total:!!$100,000 to $124,999",HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 201...,int,B19001B,0,NaN,NaN,"B19001B_014EA,B19001B_014M,B19001B_014MA",NaN


In [9]:
# Comments and code in this block are from https://github.com/censusreporter/nicar20-advanced-census-python/blob/master/workshop.ipynb:
# will use (Nicar20) as citation for the above site from here on out
# values for 'group' are ACS table IDs; 
# for this data, when it's N/A, it's for other kinds of API variables.
# so let's leave those out:
short_vars = variables[~(variables['group'] == 'N/A')] 

# So now this is an easy way to get a list of all of the table IDs and their titles
short_vars[['group', 'concept']].drop_duplicates().sort_values('group').head(10) 

,group,concept
B01001_012E,B01001,SEX BY AGE
B01001A_002E,B01001A,SEX BY AGE (WHITE ALONE)
B01001B_029E,B01001B,SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)
B01001C_008E,B01001C,SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ...
B01001D_008E,B01001D,SEX BY AGE (ASIAN ALONE)
B01001E_013E,B01001E,SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ...
B01001F_001E,B01001F,SEX BY AGE (SOME OTHER RACE ALONE)
B01001G_022E,B01001G,SEX BY AGE (TWO OR MORE RACES)
B01001H_001E,B01001H,"SEX BY AGE (WHITE ALONE, NOT HISPANIC OR LATINO)"
B01001I_027E,B01001I,SEX BY AGE (HISPANIC OR LATINO)


In [10]:
# (From Nicar20)
# Use when you know which group but still need specific API variable codes
# Attributes column shows related variables you can request. The one that ends with M is the margin of error, and since we want to be responsible when we aggregate data, we'll be sure to aggregate the error as well. 
#The other two, which end with A are "annotations." 
short_vars[short_vars['group'] == 'B01001'][['label','attributes']].sort_index() 

,label,attributes
B01001_001E,Estimate!!Total:,"B01001_001EA,B01001_001M,B01001_001MA"
B01001_002E,Estimate!!Total:!!Male:,"B01001_002EA,B01001_002M,B01001_002MA"
B01001_003E,Estimate!!Total:!!Male:!!Under 5 years,"B01001_003EA,B01001_003M,B01001_003MA"
B01001_004E,Estimate!!Total:!!Male:!!5 to 9 years,"B01001_004EA,B01001_004M,B01001_004MA"
B01001_005E,Estimate!!Total:!!Male:!!10 to 14 years,"B01001_005EA,B01001_005M,B01001_005MA"
B01001_006E,Estimate!!Total:!!Male:!!15 to 17 years,"B01001_006EA,B01001_006M,B01001_006MA"
B01001_007E,Estimate!!Total:!!Male:!!18 and 19 years,"B01001_007EA,B01001_007M,B01001_007MA"
B01001_008E,Estimate!!Total:!!Male:!!20 years,"B01001_008EA,B01001_008M,B01001_008MA"
B01001_009E,Estimate!!Total:!!Male:!!21 years,"B01001_009EA,B01001_009M,B01001_009MA"
B01001_010E,Estimate!!Total:!!Male:!!22 to 24 years,"B01001_010EA,B01001_010M,B01001_010MA"


In [11]:
# search function
#sample = censusdata.search('acs5', 2019, 'concept', 'age ')
#print(sample)

#  to see list of all tables in the ACS5
# c.acs5.tables()

In [13]:
censusdata.printtable(censusdata.censustable('acs1', 2019, 'B08604'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B08604_001E  | WORKER POPULATION FOR WORKPLAC | !! Estimate Total:                                       | int  
-------------------------------------------------------------------------------------------------------------------


In [14]:
# Run Census Search to retrieve data on Baltimore City, MD (all census tracts in Baltimore City)
# ***See https://api.census.gov/data/2019/acs/acs5/groups.html  for list of variables and groups for the ACS 5-year estimates***
# ex. "B23025_005E" is "unemployment count"
# The state FIPS code for MD is 24 and the FIPS code for Balt City is 510
census_data = c.acs5.state_county_tract(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E",
                          "B23025_004E",                                                            
                          "B15003_017E",
                          "B15003_022E",                
                          "B02001_002E",
                          "B02001_003E",
                          "B02001_008E",
                          "B03001_003E",
                          "B25008_002E",
                          "B28007_001E",
                          "B25003_002E",                                    
                          "B25003_003E"),               
                          state_fips = "24",
                          county_fips = "510",
                          tract = "*")
census_pd = pd.DataFrame(census_data)
census_pd.head()      

,NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B23025_005E,B23025_004E,B15003_017E,B15003_022E,...,B02001_003E,B02001_008E,B03001_003E,B25008_002E,B28007_001E,B25003_002E,B25003_003E,state,county,tract
0,"Census Tract 2604.03, Baltimore city, Maryland",31625.0,1764.0,27.9,21744.0,436.0,34.0,885.0,450.0,91.0,...,1741.0,0.0,0.0,306.0,1218.0,119.0,622.0,24,510,260403
1,"Census Tract 2302, Baltimore city, Maryland",107391.0,1982.0,30.7,63973.0,151.0,17.0,1500.0,72.0,593.0,...,169.0,99.0,88.0,1030.0,1808.0,539.0,443.0,24,510,230200
2,"Census Tract 2601.02, Baltimore city, Maryland",62586.0,4775.0,41.3,25903.0,469.0,151.0,2366.0,1231.0,321.0,...,4254.0,0.0,94.0,3674.0,3812.0,1384.0,323.0,24,510,260102
3,"Census Tract 2603.03, Baltimore city, Maryland",17000.0,969.0,37.0,19562.0,377.0,20.0,286.0,229.0,47.0,...,626.0,29.0,66.0,164.0,716.0,88.0,368.0,24,510,260303
4,"Census Tract 2608, Baltimore city, Maryland",40795.0,2332.0,36.1,26895.0,481.0,38.0,1233.0,394.0,309.0,...,310.0,10.0,970.0,751.0,1875.0,415.0,580.0,24,510,260800


In [27]:
age_data = c.acs5.state_county_tract(("NAME", "B01001_003E",                                    
                          "B01001_004E",
                          "B01001_005E",
                          "B01001_006E",
                          "B01001_007E",
                          "B01001_008E",
                          "B01001_009E",
                          "B01001_010E",
                          "B01001_011E",
                          "B01001_012E",
                          "B01001_013E",
                          "B01001_014E",
                          "B01001_015E",
                          "B01001_016E",            
                          "B01001_017E",
                          "B01001_018E",
                          "B01001_019E",
                          "B01001_020E",
                          "B01001_021E",
                          "B01001_022E",
                          "B01001_023E",
                          "B01001_024E",
                          "B01001_025E",
                          
                          "B01001_027E",
                          "B01001_028E",
                          "B01001_029E",
                          "B01001_030E",
                          "B01001_031E",
                          "B01001_032E",
                          "B01001_033E",
                          "B01001_034E",
                          "B01001_035E",
                          "B01001_036E",
                          "B01001_037E",
                          "B01001_038E",
                          "B01001_039E",            
                          "B01001_040E",
                          "B01001_041E",
                          "B01001_042E",
                          "B01001_043E",
                          "B01001_044E",
                          "B01001_045E",
                          "B01001_046E",
                          "B01001_047E",
                          "B01001_048E",
                          "B01001_049E"),          
                                     
                          state_fips = "24",
                          county_fips = "510",
                          tract = "*")
age_pd = pd.DataFrame(age_data)
age_pd.head()      

,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_043E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,county,tract
0,"Census Tract 2604.03, Baltimore city, Maryland",51.0,164.0,114.0,22.0,12.0,0.0,13.0,18.0,52.0,...,41.0,13.0,17.0,5.0,26.0,0.0,0.0,24,510,260403
1,"Census Tract 2302, Baltimore city, Maryland",52.0,0.0,15.0,14.0,0.0,0.0,0.0,107.0,163.0,...,15.0,22.0,18.0,11.0,23.0,11.0,32.0,24,510,230200
2,"Census Tract 2601.02, Baltimore city, Maryland",58.0,157.0,194.0,142.0,61.0,50.0,0.0,36.0,97.0,...,147.0,116.0,49.0,64.0,57.0,23.0,45.0,24,510,260102
3,"Census Tract 2603.03, Baltimore city, Maryland",66.0,49.0,0.0,11.0,0.0,23.0,0.0,23.0,20.0,...,18.0,20.0,6.0,0.0,7.0,11.0,0.0,24,510,260303
4,"Census Tract 2608, Baltimore city, Maryland",79.0,53.0,0.0,3.0,10.0,0.0,16.0,58.0,73.0,...,33.0,14.0,9.0,28.0,34.0,32.0,25.0,24,510,260800


In [31]:
# Sum columns of age groups that are under 15 years old for male and female and add new column "Pop. under 15 years"
columns_under15 = ["B01001_003E",                                    
                   "B01001_004E",
                   "B01001_005E",
                   "B01001_027E",
                   "B01001_028E",
                   "B01001_029E"]
age_pd['Pop. under 15 years']= age_pd[columns_under15].sum(axis=1)
age_pd.head()

,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,county,tract,Under 15 years
0,"Census Tract 2604.03, Baltimore city, Maryland",51.0,164.0,114.0,22.0,12.0,0.0,13.0,18.0,52.0,...,13.0,17.0,5.0,26.0,0.0,0.0,24,510,260403,505.0
1,"Census Tract 2302, Baltimore city, Maryland",52.0,0.0,15.0,14.0,0.0,0.0,0.0,107.0,163.0,...,22.0,18.0,11.0,23.0,11.0,32.0,24,510,230200,160.0
2,"Census Tract 2601.02, Baltimore city, Maryland",58.0,157.0,194.0,142.0,61.0,50.0,0.0,36.0,97.0,...,116.0,49.0,64.0,57.0,23.0,45.0,24,510,260102,863.0
3,"Census Tract 2603.03, Baltimore city, Maryland",66.0,49.0,0.0,11.0,0.0,23.0,0.0,23.0,20.0,...,20.0,6.0,0.0,7.0,11.0,0.0,24,510,260303,249.0
4,"Census Tract 2608, Baltimore city, Maryland",79.0,53.0,0.0,3.0,10.0,0.0,16.0,58.0,73.0,...,14.0,9.0,28.0,34.0,32.0,25.0,24,510,260800,430.0


In [32]:
# Sum columns of age groups that are 15-74 years old (working age) for male and female and add new column "Pop. under 15 years"
columns_working_age = ["B01001_006E",                                    
                   "B01001_007E",
                   "B01001_008E",
                   "B01001_009E",
                   "B01001_010E",
                   "B01001_011E",
                   "B01001_012E",   
                   "B01001_013E",
                   "B01001_014E",    
                   "B01001_015E",    
                   "B01001_016E",   
                   "B01001_017E", 
                   "B01001_018E",    
                   "B01001_019E",    
                   "B01001_020E",    
                   "B01001_021E",
                   "B01001_022E",
                   "B01001_030E",
                   "B01001_031E",
                   "B01001_032E",
                   "B01001_033E",
                   "B01001_034E",
                   "B01001_035E",   
                   "B01001_036E",
                   "B01001_037E",    
                   "B01001_038E",    
                   "B01001_039E",   
                   "B01001_040E", 
                   "B01001_041E",    
                   "B01001_042E",    
                   "B01001_043E",    
                   "B01001_044E",
                   "B01001_045E",    
                   "B01001_046E"]         
age_pd['Pop. working age']= age_pd[columns_working_age].sum(axis=1)
age_pd.head()                
              
               


,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,county,tract,Under 15 years,Pop. working age
0,"Census Tract 2604.03, Baltimore city, Maryland",51.0,164.0,114.0,22.0,12.0,0.0,13.0,18.0,52.0,...,17.0,5.0,26.0,0.0,0.0,24,510,260403,505.0,1233.0
1,"Census Tract 2302, Baltimore city, Maryland",52.0,0.0,15.0,14.0,0.0,0.0,0.0,107.0,163.0,...,18.0,11.0,23.0,11.0,32.0,24,510,230200,160.0,1716.0
2,"Census Tract 2601.02, Baltimore city, Maryland",58.0,157.0,194.0,142.0,61.0,50.0,0.0,36.0,97.0,...,49.0,64.0,57.0,23.0,45.0,24,510,260102,863.0,3718.0
3,"Census Tract 2603.03, Baltimore city, Maryland",66.0,49.0,0.0,11.0,0.0,23.0,0.0,23.0,20.0,...,6.0,0.0,7.0,11.0,0.0,24,510,260303,249.0,691.0
4,"Census Tract 2608, Baltimore city, Maryland",79.0,53.0,0.0,3.0,10.0,0.0,16.0,58.0,73.0,...,9.0,28.0,34.0,32.0,25.0,24,510,260800,430.0,1783.0


In [34]:
# Sum columns of age groups that are 75+ years old for male and female and add new column "Pop. over 74 years"
columns_senior = ["B01001_023E",                                    
                  "B01001_024E",
                  "B01001_025E",
                  "B01001_047E",                                    
                  "B01001_048E",
                  "B01001_049E"]               
                          
age_pd['Pop. over 74 years']= age_pd[columns_senior].sum(axis=1)
age_pd.head()                           

,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,county,tract,Under 15 years,Pop. working age,Pop. over 74 years
0,"Census Tract 2604.03, Baltimore city, Maryland",51.0,164.0,114.0,22.0,12.0,0.0,13.0,18.0,52.0,...,5.0,26.0,0.0,0.0,24,510,260403,505.0,1233.0,26.0
1,"Census Tract 2302, Baltimore city, Maryland",52.0,0.0,15.0,14.0,0.0,0.0,0.0,107.0,163.0,...,11.0,23.0,11.0,32.0,24,510,230200,160.0,1716.0,106.0
2,"Census Tract 2601.02, Baltimore city, Maryland",58.0,157.0,194.0,142.0,61.0,50.0,0.0,36.0,97.0,...,64.0,57.0,23.0,45.0,24,510,260102,863.0,3718.0,194.0
3,"Census Tract 2603.03, Baltimore city, Maryland",66.0,49.0,0.0,11.0,0.0,23.0,0.0,23.0,20.0,...,0.0,7.0,11.0,0.0,24,510,260303,249.0,691.0,29.0
4,"Census Tract 2608, Baltimore city, Maryland",79.0,53.0,0.0,3.0,10.0,0.0,16.0,58.0,73.0,...,28.0,34.0,32.0,25.0,24,510,260800,430.0,1783.0,119.0


In [39]:
age_final = age_pd[[ "tract", "Under 15 years", "Pop. working age", "Pop. over 74 years"]]
age_final.head()

,tract,Under 15 years,Pop. working age,Pop. over 74 years
0,260403,505.0,1233.0,26.0
1,230200,160.0,1716.0,106.0
2,260102,863.0,3718.0,194.0
3,260303,249.0,691.0,29.0
4,260800,430.0,1783.0,119.0


In [15]:
# did not add in daytime population - will use ESRI business analyst for this 
# See https://www.census.gov/topics/employment/commuting/guidance/calculations.html
# "commuter-adjusted daytime population estimates" =    
#         total resident population + total workers working in area - total workers living in area

# For "Workers in Workplace Geography," see https://www.census.gov/topics/employment/commuting/guidance/calculations.html
# "Total workers working in area:
# B08604 Total Workers for Workplace Geography
# B08604 is only available for data years 2011 and after. 
# The tables for workplace geography are only available for the following geographic summary levels: States; 
# Counties; Places; County Subdivisions in selected states (not MD); Combined Statistical Areas; Metropolitan 
# and Micropolitan Statistical Areas, and their associated Metropolitan Divisions and Principal Cities; 

census_data_workers = c.acs5.state_county(("NAME", 
                          "B08604_001E"),               
                          state_fips = "24",
                          county_fips = "510") 

# convert to dataframe
workers_df = pd.DataFrame(census_data_workers)
workers_df
                         

,NAME,B08604_001E,state,county
0,"Baltimore city, Maryland",383366.0,24,510


In [16]:
# Create Geographic Identifier ("GEOID") for each census tract by adding state fips code + county fips code + census tract #
# see https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html#:~:text=The%20full%20GEOID%20for%20many,codes%2C%20in%20which%20they%20nest.
census_pd["GEOID"] = census_pd['state'] + census_pd['county'] + census_pd['tract']
census_pd.head()

,NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B23025_005E,B23025_004E,B15003_017E,B15003_022E,...,B02001_008E,B03001_003E,B25008_002E,B28007_001E,B25003_002E,B25003_003E,state,county,tract,GEOID
0,"Census Tract 2604.03, Baltimore city, Maryland",31625.0,1764.0,27.9,21744.0,436.0,34.0,885.0,450.0,91.0,...,0.0,0.0,306.0,1218.0,119.0,622.0,24,510,260403,24510260403
1,"Census Tract 2302, Baltimore city, Maryland",107391.0,1982.0,30.7,63973.0,151.0,17.0,1500.0,72.0,593.0,...,99.0,88.0,1030.0,1808.0,539.0,443.0,24,510,230200,24510230200
2,"Census Tract 2601.02, Baltimore city, Maryland",62586.0,4775.0,41.3,25903.0,469.0,151.0,2366.0,1231.0,321.0,...,0.0,94.0,3674.0,3812.0,1384.0,323.0,24,510,260102,24510260102
3,"Census Tract 2603.03, Baltimore city, Maryland",17000.0,969.0,37.0,19562.0,377.0,20.0,286.0,229.0,47.0,...,29.0,66.0,164.0,716.0,88.0,368.0,24,510,260303,24510260303
4,"Census Tract 2608, Baltimore city, Maryland",40795.0,2332.0,36.1,26895.0,481.0,38.0,1233.0,394.0,309.0,...,10.0,970.0,751.0,1875.0,415.0,580.0,24,510,260800,24510260800


In [17]:
# number of rows = # of census tracts in the dataframe
print("Number of rows, columns: ", census_pd.shape)

Number of rows, columns:  (200, 22)


In [44]:
# combine age data with rest of census data
census_joined = pd.concat([age_final, census_pd], axis="columns")
census_joined.head()

,tract,Under 15 years,Pop. working age,Pop. over 74 years,State,Name,GEOID,Population,Median household income,Per capita income,...,# persons age 25+ with Bachelor's degree,Median age,Pop. white,Pop. Black,Pop. 2 or more races,Pop. Hispanic origin,Total pop. in occupied housing units by tenure,Total owner-occupied units,Total renter-occupied units,Labor force status by presence of computer/ internet in HH
0,260403,505.0,1233.0,26.0,24,"Census Tract 2604.03, Baltimore city, Maryland",24510260403,1764.0,31625.0,21744.0,...,91.0,27.9,23.0,1741.0,0.0,0.0,306.0,119.0,622.0,1218.0
1,230200,160.0,1716.0,106.0,24,"Census Tract 2302, Baltimore city, Maryland",24510230200,1982.0,107391.0,63973.0,...,593.0,30.7,1683.0,169.0,99.0,88.0,1030.0,539.0,443.0,1808.0
2,260102,863.0,3718.0,194.0,24,"Census Tract 2601.02, Baltimore city, Maryland",24510260102,4775.0,62586.0,25903.0,...,321.0,41.3,440.0,4254.0,0.0,94.0,3674.0,1384.0,323.0,3812.0
3,260303,249.0,691.0,29.0,24,"Census Tract 2603.03, Baltimore city, Maryland",24510260303,969.0,17000.0,19562.0,...,47.0,37.0,281.0,626.0,29.0,66.0,164.0,88.0,368.0,716.0
4,260800,430.0,1783.0,119.0,24,"Census Tract 2608, Baltimore city, Maryland",24510260800,2332.0,40795.0,26895.0,...,309.0,36.1,1748.0,310.0,10.0,970.0,751.0,415.0,580.0,1875.0


In [48]:
# Replace the census variable codes (such as "B19013_001E") in the dataframe with text so it's more easily understandable
census_joined = census_joined.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median age",
                                      "B19013_001E": "Median household income",
                                      "B19301_001E": "Per capita income", 
                                      "B17001_002E": "Poverty count",
                                      "B23025_004E": "# employed, age 16+",
                                      "B23025_005E": "Unemployment count",
                                      "B15003_017E": "# persons age 25+ graduated high school",
                                      "B15003_022E": "# persons age 25+ with Bachelor's degree",
                                      "B02001_002E": "Pop. white",
                                      "B02001_003E": "Pop. Black",
                                      "B02001_008E": "Pop. 2 or more races",
                                      "B03001_003E": "Pop. Hispanic origin",
                                      "B25008_002E": "Total pop. in occupied housing units by tenure",
                                      "B25003_002E": "Total owner-occupied units",
                                      "B25003_003E": "Total renter-occupied units",
                                      "Under 15 years": "Pop. under 15 years",
                                      "B28007_001E": "Labor force status by presence of computer/ internet in HH",
                                      "NAME": "Name", "state": "State", "GEOID": "GEOID"})

# Add a new column for poverty rate (Poverty Count / Population)
census_joined["Poverty rate"] = 100 * \
    census_joined["Poverty count"].astype(
        int) / census_joined["Population"].astype(int)


# Add a new column for unemployment rate (Employment Count / Population)
census_joined["Unemployment rate"] = 100 * \
    census_joined["Unemployment count"].astype(
        int) / census_joined["Population"].astype(int)
census_joined.head()

,tract,Pop. under 15 years,Pop. working age,Pop. over 74 years,State,Name,GEOID,Population,Median household income,Per capita income,...,# persons age 25+ with Bachelor's degree,Median age,Pop. white,Pop. Black,Pop. 2 or more races,Pop. Hispanic origin,Total pop. in occupied housing units by tenure,Total owner-occupied units,Total renter-occupied units,Labor force status by presence of computer/ internet in HH
0,260403,505.0,1233.0,26.0,24,"Census Tract 2604.03, Baltimore city, Maryland",24510260403,1764.0,31625.0,21744.0,...,91.0,27.9,23.0,1741.0,0.0,0.0,306.0,119.0,622.0,1218.0
1,230200,160.0,1716.0,106.0,24,"Census Tract 2302, Baltimore city, Maryland",24510230200,1982.0,107391.0,63973.0,...,593.0,30.7,1683.0,169.0,99.0,88.0,1030.0,539.0,443.0,1808.0
2,260102,863.0,3718.0,194.0,24,"Census Tract 2601.02, Baltimore city, Maryland",24510260102,4775.0,62586.0,25903.0,...,321.0,41.3,440.0,4254.0,0.0,94.0,3674.0,1384.0,323.0,3812.0
3,260303,249.0,691.0,29.0,24,"Census Tract 2603.03, Baltimore city, Maryland",24510260303,969.0,17000.0,19562.0,...,47.0,37.0,281.0,626.0,29.0,66.0,164.0,88.0,368.0,716.0
4,260800,430.0,1783.0,119.0,24,"Census Tract 2608, Baltimore city, Maryland",24510260800,2332.0,40795.0,26895.0,...,309.0,36.1,1748.0,310.0,10.0,970.0,751.0,415.0,580.0,1875.0


In [49]:
# number of rows = # of census tracts in the dataframe
print("Number of rows, columns: ", census_joined.shape)

Number of rows, columns:  (200, 26)


In [52]:
# Add in home ownership rate (# owner-occupied units / # of occupied housing units)
# sum 2 columns: total owner-occupied units + total renter-occupied units to create additional column "Total occupied units" 
sum_column = census_joined['Total owner-occupied units'] + census_joined['Total renter-occupied units']
census_joined["Total occupied units"] = sum_column

In [20]:
census_joined["Home ownership rate"] = 100 * \
    census_joined["Total owner-occupied units"].astype(
        int) / census_joined["Total occupied units"].astype(
        int) 

census_joined.head()

,Name,Median household income,Population,Median age,Per capita income,Poverty count,Unemployment count,"# employed, age 16+",# persons age 25+ graduated high school,# persons age 25+ with Bachelor's degree,...,Total owner-occupied units,Total renter-occupied units,State,county,tract,GEOID,Poverty rate,Unemployment rate,Total occupied units,Home ownership rate
0,"Census Tract 2604.03, Baltimore city, Maryland",31625.0,1764.0,27.9,21744.0,436.0,34.0,885.0,450.0,91.0,...,119.0,622.0,24,510,260403,24510260403,24.716553,1.927438,741.0,16.059379
1,"Census Tract 2302, Baltimore city, Maryland",107391.0,1982.0,30.7,63973.0,151.0,17.0,1500.0,72.0,593.0,...,539.0,443.0,24,510,230200,24510230200,7.618567,0.857719,982.0,54.887984
2,"Census Tract 2601.02, Baltimore city, Maryland",62586.0,4775.0,41.3,25903.0,469.0,151.0,2366.0,1231.0,321.0,...,1384.0,323.0,24,510,260102,24510260102,9.821990,3.162304,1707.0,81.077914
3,"Census Tract 2603.03, Baltimore city, Maryland",17000.0,969.0,37.0,19562.0,377.0,20.0,286.0,229.0,47.0,...,88.0,368.0,24,510,260303,24510260303,38.906089,2.063983,456.0,19.298246
4,"Census Tract 2608, Baltimore city, Maryland",40795.0,2332.0,36.1,26895.0,481.0,38.0,1233.0,394.0,309.0,...,415.0,580.0,24,510,260800,24510260800,20.626072,1.629503,995.0,41.708543


In [53]:
# round the home ownership rate to one decimal point; using "float" instead of "int" because want to use decimal points
census_joined["Home ownership rate"] = census_joined["Home ownership rate"].astype(float).round(1)


KeyError: 'Home ownership rate'

In [22]:
census_joined["Poverty rate"] = census_joined["Poverty rate"].astype(float).round(1)


In [23]:
census_joined["Unemployment rate"] = census_joined["Unemployment rate"].astype(float).round(1)
census_joined.head()

,Name,Median household income,Population,Median age,Per capita income,Poverty count,Unemployment count,"# employed, age 16+",# persons age 25+ graduated high school,# persons age 25+ with Bachelor's degree,...,Total owner-occupied units,Total renter-occupied units,State,county,tract,GEOID,Poverty rate,Unemployment rate,Total occupied units,Home ownership rate
0,"Census Tract 2604.03, Baltimore city, Maryland",31625.0,1764.0,27.9,21744.0,436.0,34.0,885.0,450.0,91.0,...,119.0,622.0,24,510,260403,24510260403,24.7,1.9,741.0,16.1
1,"Census Tract 2302, Baltimore city, Maryland",107391.0,1982.0,30.7,63973.0,151.0,17.0,1500.0,72.0,593.0,...,539.0,443.0,24,510,230200,24510230200,7.6,0.9,982.0,54.9
2,"Census Tract 2601.02, Baltimore city, Maryland",62586.0,4775.0,41.3,25903.0,469.0,151.0,2366.0,1231.0,321.0,...,1384.0,323.0,24,510,260102,24510260102,9.8,3.2,1707.0,81.1
3,"Census Tract 2603.03, Baltimore city, Maryland",17000.0,969.0,37.0,19562.0,377.0,20.0,286.0,229.0,47.0,...,88.0,368.0,24,510,260303,24510260303,38.9,2.1,456.0,19.3
4,"Census Tract 2608, Baltimore city, Maryland",40795.0,2332.0,36.1,26895.0,481.0,38.0,1233.0,394.0,309.0,...,415.0,580.0,24,510,260800,24510260800,20.6,1.6,995.0,41.7


In [24]:
# Calculate population density  see: https://www.census.gov/quickfacts/fact/note/US/LND110210
# density is expressed as "population per square mile(kilometer)"
# Divide total population (or # of housing units)/ by land area of the entity measured in square miles

In [25]:
# Change order of columns in DataFrame by using double brackets
census_joined = census_joined[["State", "Name", "GEOID", "Population", "Median household income",
                       "Per capita income", "Poverty count", "Poverty rate", "Unemployment rate", 
                       "# employed, age 16+", "Unemployment count",
                      "# persons age 25+ graduated high school", "# persons age 25+ with Bachelor's degree",
                      "Median age","Pop. white", "Pop. Black", "Pop. 2 or more races", "Pop. Hispanic origin", 
                      "Total pop. in occupied housing units by tenure", "Total owner-occupied units", "Total renter-occupied units",
                      "Pop. under 15 years", "Pop. working age", "Pop. over 74 years",         
                      "Labor force status by presence of computer/ internet in HH"]]

census_joined.head()

,State,Name,GEOID,Population,Median household income,Per capita income,Poverty count,Poverty rate,Unemployment rate,"# employed, age 16+",...,# persons age 25+ with Bachelor's degree,Median age,Pop. white,Pop. Black,Pop. 2 or more races,Pop. Hispanic origin,Total pop. in occupied housing units by tenure,Total owner-occupied units,Total renter-occupied units,Labor force status by presence of computer/ internet in HH
0,24,"Census Tract 2604.03, Baltimore city, Maryland",24510260403,1764.0,31625.0,21744.0,436.0,24.7,1.9,885.0,...,91.0,27.9,23.0,1741.0,0.0,0.0,306.0,119.0,622.0,1218.0
1,24,"Census Tract 2302, Baltimore city, Maryland",24510230200,1982.0,107391.0,63973.0,151.0,7.6,0.9,1500.0,...,593.0,30.7,1683.0,169.0,99.0,88.0,1030.0,539.0,443.0,1808.0
2,24,"Census Tract 2601.02, Baltimore city, Maryland",24510260102,4775.0,62586.0,25903.0,469.0,9.8,3.2,2366.0,...,321.0,41.3,440.0,4254.0,0.0,94.0,3674.0,1384.0,323.0,3812.0
3,24,"Census Tract 2603.03, Baltimore city, Maryland",24510260303,969.0,17000.0,19562.0,377.0,38.9,2.1,286.0,...,47.0,37.0,281.0,626.0,29.0,66.0,164.0,88.0,368.0,716.0
4,24,"Census Tract 2608, Baltimore city, Maryland",24510260800,2332.0,40795.0,26895.0,481.0,20.6,1.6,1233.0,...,309.0,36.1,1748.0,310.0,10.0,970.0,751.0,415.0,580.0,1875.0


KeyError: "None of [Index(['tract', 'B01001_001E', 'B01001_001M'], dtype='object')] are in the [columns]"

In [ ]:
census_joined.count()

In [ ]:
# Split the "Name" column into 3 separate columns: "Census_Tract", "County", "State"
census_joined[['Census_Tract', "County", "State"]]= census_joine['Name'].str.split(",", n=3, expand=True)
census_joined.head()

In [ ]:
# Remove "State" column because it's understood we are looking at MD data
census_joined = census_joined.drop(["State"], axis=1)


In [ ]:
# list the columns in the census_pd dataframe
census_joined.columns

In [ ]:
# Create new column "Census_Tract" and remove the text "Census Tract" from the values in that column (to make calculations easier)
census_joined["Census_Tract"] = census_joined['Census_Tract'].str.replace('Census Tract', "") 


In [ ]:
# Calculate the number of unique census tracts in the DataFrame
tract_count = len(census_joined["Census_Tract"].unique())
tract_count

In [ ]:
census_final = census_joined.drop(["Name"], axis=1)
census_final.head()

In [ ]:
# Calculations can also be performed on Series and added into DataFrames as new columns
#thousands_of_dollars = data_file_df["Amount"]/1000
#data_file_df["Thousands of Dollars"] = thousands_of_dollars

#data_file_df.head()

In [ ]:
# Converting the membership days into weeks and then adding a column to the DataFrame
#weeks = training_df["Membership (Days)"]/7
#training_df["Membership (Weeks)"] = weeks

#training_df.head()

In [ ]:
# Export file as a CSV, without the Pandas index, but with the header
#census_pd.to_csv("Comm_Corridor_Stats_Draft2.csv", index = False, header=True)